In [162]:
import pandas as pd

In [163]:
stats = pd.read_csv('mvp_stats.csv')

In [164]:
del stats['Unnamed: 0']

In [165]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          58
3P            0
3PA           0
3P%        1886
2P            0
2PA           0
2P%         103
eFG%         58
FT            0
FTA           0
FT%         549
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W           572
L           572
W/L%        572
GB            0
PS/G        572
PA/G        572
SRS         572
dtype: int64

In [166]:
stats = stats.fillna(0)

In [167]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

In [168]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']

In [169]:
train = stats[stats['Year'] < 2024]

In [170]:
test = stats[stats['Year'] == 2024]

In [171]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=0.1)

In [172]:
reg.fit(train[predictors], train['Share'])

Ridge(alpha=0.1)

In [173]:
predictions = reg.predict(test[predictors])

In [174]:
predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)

In [175]:
predictions

,Predictions
467,0.043355
468,0.070882
469,0.064254
470,0.075223
471,0.066077
...,...
14239,0.070671
14240,0.062644
14241,0.043851
14242,0.079572


In [176]:
combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)

In [177]:
combine


,Player,Share,Predictions
467,AJ Griffin,0.0,0.043355
468,Bogdan Bogdanović,0.0,0.070882
469,Bruno Fernando,0.0,0.064254
470,Clint Capela,0.0,0.075223
471,De'Andre Hunter,0.0,0.066077
...,...,...,...
14239,Marvin Bagley III,0.0,0.070671
14240,Patrick Baldwin Jr.,0.0,0.062644
14241,Richaun Holmes,0.0,0.043851
14242,Tristan Vukcevic,0.0,0.079572


In [178]:
combine.sort_values(by='Share', ascending=False).head(10)

,Player,Share,Predictions
3556,Nikola Jokić,0.935,0.222206
9654,Shai Gilgeous-Alexander,0.646,0.222172
3081,Luka Dončić,0.572,0.247123
7782,Giannis Antetokounmpo,0.194,0.271961
9370,Jalen Brunson,0.143,0.153325
950,Jayson Tatum,0.087,0.168258
8238,Anthony Edwards,0.018,0.141886
12011,Domantas Sabonis,0.003,0.159343
11082,Kevin Durant,0.001,0.159826
9643,Isaiah Joe,0.000,0.064032


In [179]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combine['Share'], combine['Predictions'])

0.00631199754710459

In [180]:
combine['Share'].value_counts()

Share
0.000    563
0.087      1
0.572      1
0.935      1
0.194      1
0.018      1
0.143      1
0.646      1
0.001      1
0.003      1
Name: count, dtype: int64

In [181]:
combine = combine.sort_values(by='Share', ascending=False)
combine['Rk'] = list(range(1, combine.shape[0] + 1))

In [182]:
combine.head(10)

,Player,Share,Predictions,Rk
3556,Nikola Jokić,0.935,0.222206,1
9654,Shai Gilgeous-Alexander,0.646,0.222172,2
3081,Luka Dončić,0.572,0.247123,3
7782,Giannis Antetokounmpo,0.194,0.271961,4
9370,Jalen Brunson,0.143,0.153325,5
950,Jayson Tatum,0.087,0.168258,6
8238,Anthony Edwards,0.018,0.141886,7
12011,Domantas Sabonis,0.003,0.159343,8
11082,Kevin Durant,0.001,0.159826,9
9643,Isaiah Joe,0.000,0.064032,10


In [183]:
combine = combine.sort_values(by='Predictions', ascending=False)
combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))

In [184]:
combine.head(10)

,Player,Share,Predictions,Rk,Predicted_Rk
7782,Giannis Antetokounmpo,0.194,0.271961,4,1
10612,Joel Embiid,0.000,0.256540,45,2
3081,Luka Dončić,0.572,0.247123,3,3
3556,Nikola Jokić,0.935,0.222206,1,4
9654,Shai Gilgeous-Alexander,0.646,0.222172,2,5
6420,LeBron James,0.000,0.189309,477,6
6408,Anthony Davis,0.000,0.189024,439,7
950,Jayson Tatum,0.087,0.168258,6,8
12502,Victor Wembanyama,0.000,0.163888,173,9
11082,Kevin Durant,0.001,0.159826,9,10


In [185]:
combine.sort_values(by='Share', ascending=False).head(10)

,Player,Share,Predictions,Rk,Predicted_Rk
3556,Nikola Jokić,0.935,0.222206,1,4
9654,Shai Gilgeous-Alexander,0.646,0.222172,2,5
3081,Luka Dončić,0.572,0.247123,3,3
7782,Giannis Antetokounmpo,0.194,0.271961,4,1
9370,Jalen Brunson,0.143,0.153325,5,14
950,Jayson Tatum,0.087,0.168258,6,8
8238,Anthony Edwards,0.018,0.141886,7,19
12011,Domantas Sabonis,0.003,0.159343,8,12
11082,Kevin Durant,0.001,0.159826,9,10
8869,Herbert Jones,0.000,0.052664,118,388


In [186]:
def find_ap(combine):
    actual = combine.sort_values(by='Share', ascending=False).head(5)
    predicted = combine.sort_values(by='Predictions', ascending=False)
    ps = []
    found = 0
    seen = 1
    for i, r in predicted.iterrows():
        if r['Player'] in actual['Player'].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps)/len(ps)

In [187]:
find_ap(combine)

0.7147619047619048

In [188]:
years = list(range(1995, 2025))

In [189]:
aps = []
all_preds = []
for year in years[5:]:
    train = stats[stats['Year'] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha=0.1)
    reg.fit(train[predictors], train['Share'])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
    combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)
    all_preds.append(combine)
    aps.append(find_ap(combine))

In [190]:
sum(aps)/len(aps)

0.7394186400549154

In [191]:
def add_ranks(combine):
    combine = combine.sort_values(by='Share', ascending=False)
    combine['Rk'] = list(range(1, combine.shape[0] + 1))
    combine = combine.sort_values(by='Predictions', ascending=False)
    combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))
    combine['Difference'] = combine['Rk'] - combine['Predicted_Rk']
    return combine

In [192]:
ranking = add_ranks(all_preds[1])
ranking[ranking['Rk'] < 6].sort_values(by='Difference', ascending=False)

,Player,Share,Predictions,Rk,Predicted_Rk,Difference
6047,Shaquille O'Neal,0.466,0.272036,3,1,2
11647,Chris Webber,0.420,0.157423,4,2,2
12128,Tim Duncan,0.569,0.137012,2,3,-1
7887,Kevin Garnett,0.122,0.115111,5,7,-2
10227,Allen Iverson,0.904,0.123538,1,5,-4


In [193]:
def backtest(stats, model, year, predictors):
    aps = []
    all_preds = []
    for year in years[5:]:
        train = stats[stats['Year'] < year]
        test = stats[stats['Year'] == year]
        model.fit(train[predictors], train['Share'])
        predictions = model.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
        combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)
        combine = add_ranks(combine)
        all_preds.append(combine)
        aps.append(find_ap(combine))
    return sum(aps)/len(aps), aps, pd.concat(all_preds)

In [194]:
mean_ap, aps, all_preds = backtest(stats,reg, years[5:], predictors)

In [195]:
mean_ap

0.7394186400549154

In [196]:
all_preds[all_preds['Rk']<=5].sort_values(by='Difference', ascending=False)

,Player,Share,Predictions,Rk,Predicted_Rk,Difference
8704,Anthony Davis,0.156,0.147815,5,1,4
6074,Shaquille O'Neal,0.106,0.214383,5,1,4
7725,Giannis Antetokounmpo,0.345,0.203472,4,1,3
2294,LeBron James,0.142,0.144979,5,2,3
12218,Tim Duncan,0.222,0.153712,4,1,3
...,...,...,...,...,...,...
10799,Steve Nash,0.739,0.060247,1,31,-30
11695,Peja Stojaković,0.228,0.034305,4,39,-35
1919,Joakim Noah,0.258,0.044454,4,40,-36
10781,Steve Nash,0.839,0.041184,1,41,-40


In [197]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(by=0, ascending=False)

,0,1
13,0.118759,eFG%
18,0.029277,DRB
29,0.028873,W/L%
17,0.016694,ORB
10,0.014796,2P
15,0.011408,FTA
12,0.010685,2P%
21,0.010172,STL
25,0.008678,PTS
22,0.007959,BLK


In [198]:
ratios = stats[['PTS', 'AST', 'STL', 'BLK', '3P', 'Year']].apply(lambda x: x/x.mean())

In [199]:
ratios

,PTS,AST,STL,BLK,3P,Year
0,1.177563,0.489219,0.777978,4.390800,0.000000,0.992408
1,1.177563,1.250227,1.400360,0.243933,1.608074,0.992408
2,0.218517,0.163073,0.155596,0.243933,0.000000,0.992408
3,0.315636,1.087154,1.089169,0.000000,0.160807,0.992408
4,0.097119,0.108715,0.000000,0.000000,0.321615,0.992408
...,...,...,...,...,...,...
14239,1.420360,0.597935,0.622382,1.707533,0.321615,1.006834
14240,0.534152,0.434862,0.777978,0.975733,1.286459,1.006834
14241,0.606991,0.326146,0.466787,1.219667,0.000000,1.006834
14242,1.031885,0.706650,0.777978,1.707533,1.608074,1.006834


In [200]:
stats[['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']] = ratios[['PTS', 'AST', 'STL', 'BLK', '3P']]

In [201]:
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,W/L%,GB,PS/G,PA/G,SRS,PTS_T,AST_R,STL_R,BLK_R,3P_R
0,Andrew Lang,C,28,ATL,82,63,28.5,3.9,8.3,0.473,...,0.512,10.0,96.6,95.3,1.06,1.177563,0.489219,0.777978,4.390800,0.000000
1,Craig Ehlo,SG,33,ATL,49,0,23.8,3.9,8.6,0.453,...,0.512,10.0,96.6,95.3,1.06,1.177563,1.250227,1.400360,0.243933,1.608074
2,Doug Edwards,PF,24,ATL,38,0,5.6,0.6,1.3,0.458,...,0.512,10.0,96.6,95.3,1.06,0.218517,0.163073,0.155596,0.243933,0.000000
3,Ennis Whatley,PG,32,ATL,27,2,10.8,0.9,2.0,0.453,...,0.512,10.0,96.6,95.3,1.06,0.315636,1.087154,1.089169,0.000000,0.160807
4,Fred Vinson,SG,24,ATL,5,0,5.4,0.2,1.4,0.143,...,0.512,10.0,96.6,95.3,1.06,0.097119,0.108715,0.000000,0.000000,0.321615


In [202]:
predictors+=['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']

In [203]:
mean_ap, aps, all_preds = backtest(stats, reg, years[5:], predictors)

In [204]:
mean_ap

0.7394186400549154

In [205]:
stats['NPos'] = stats['Pos'].astype('category').cat.codes

In [206]:
stats['NTm'] = stats['Tm'].astype('category').cat.codes

In [207]:
stats['NTm'].value_counts()    

NTm
7     498
27    495
17    495
8     489
14    487
0     485
33    483
12    482
31    478
15    477
1     476
13    475
18    475
11    475
26    474
9     473
29    472
5     472
10    471
24    467
30    465
28    465
34    464
19    461
36    449
16    397
20    286
25    274
2     220
32    209
23    198
6     169
3     157
21    143
4     121
35     88
37     47
22     32
Name: count, dtype: int64

In [212]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, random_state=1, min_samples_split=10)

mean_ap, aps, all_preds = backtest(stats, rf, years[5:], predictors)

In [213]:
mean_ap

0.7249732097967392

In [214]:
mean_ap, aps, all_preds = backtest(stats, reg, years[5:], predictors)

In [215]:
mean_ap

0.7394186400549154